In [ ]:
from joblib import delayed

from BenchmarkBuilder.rule.count_SAQ import CountSAQGenerator
from BenchmarkBuilder.rule.count_TFQ import CountTFQGenerator
from BenchmarkBuilder.rule.distance_SAQ import DistanceSAQGenerator
from BenchmarkBuilder.rule.distance_TFQ import DistanceTFQGenerator
from BenchmarkBuilder.rule.volume_SAQ import VolumeSAQGenerator
from BenchmarkBuilder.rule.volume_TFQ import VolumeTFQGenerator
from utils.io import enum_files
from utils.parallel import ParallelTqdm

In [ ]:

def generate_questions(
        generator_class,
        scene_stat_json_file: str,
        output_json_file: str,
        n_questions: int,
):
    generator_class(
        scene_stat_json_file=scene_stat_json_file,
        output_json_file=output_json_file,
    ).generate(n_questions=n_questions)

In [ ]:
export_path = '../output'
scene_json_dir = '../NUMINA-dataset/ScanNet-scene_stats'

tasks = [
    (generator, scene_json, os.path.join(export_path, filename), 6)
    for scene_json in enum_files('../ScanNet-scene_stats')
    for generator, filename in [
        (VolumeSAQGenerator, 'NUM-volume-SAQ.json'),
        (VolumeTFQGenerator, 'NUM-volume-TFQ.json'),
        (CountSAQGenerator, 'NUM-count-SAQ.json'),
        (CountTFQGenerator, 'NUM-count-TFQ.json'),
        (DistanceSAQGenerator, 'NUM-distance-SAQ.json'),
        (DistanceTFQGenerator, 'NUM-distance-TFQ.json'),
    ]
]

ParallelTqdm(n_jobs=-1)(
    delayed(generate_questions)(generator_class, scene_json, output_json, n_questions)
    for generator_class, scene_json, output_json, n_questions in tasks
)
